# Embedding を実施してみる。

BERT を使ったテスト。

In [ ]:
import torch
from transformers import BertForPreTraining
from transformers import BertJapaneseTokenizer, BertModel

tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-v2')

## 文章をベクトル化する

In [ ]:
texts = [
    "私はラーメンと餃子が好きです。",
    "私はチャーハンを食べます。",
    "私はラーメンが好きです。",
]

def embedding(texts):
    vec = []
    for t in texts:
        encoded = tokenizer._encode_plus(t, truncation=True, padding=False, return_tensors='pt')
        input_ids = encoded["input_ids"]
    
        outputs = model(**encoded)
    
        vec.append(outputs.pooler_output.detach().numpy()[0])
    return vec

vec = embedding(texts)
vec[0].shape

## ベクトル同士のコサイン類似度を計算する

In [ ]:
import numpy as np

def comp_sim(q, t):
    return np.dot(q, t) / (np.linalg.norm(q) * np.linalg.norm(t))

In [ ]:
for i in range(len(vec)):
    for j in range(i + 1, len(vec)):
        sim = comp_sim(vec[i], vec[j])
        print(f"{i}-{j}: {sim}")

In [ ]:
texts = [
    "会社をクビになった。",
    "仕事でクビが痛い。",
    "会社を解雇された。",
]
vec = embedding(texts)

for i in range(len(vec)):
    for j in range(i + 1, len(vec)):
        sim = comp_sim(vec[i], vec[j])
        print(f"{i}-{j}: {sim}")

# Cohere Embed

In [ ]:
import os
from dotenv import load_dotenv
import cohere

load_dotenv()
co = cohere.ClientV2(os.environ['COHERE_API_KEY'])

In [ ]:
text_inputs = [
    { "content": [ {"type": "text", "text": "会社をクビになった。"}, ] },
    { "content": [ {"type": "text", "text": "仕事でクビが痛い。"}, ] },
    { "content": [ {"type": "text", "text": "会社を解雇された。"} ] },
]

response = co.embed(
    inputs=text_inputs,
    model="embed-v4.0",
    input_type="classification",
    embedding_types=["float"],
)

embeddings = response.embeddings.float

for i in range(len(embeddings)):
    for j in range(i + 1, len(embeddings)):
        sim = comp_sim(embeddings[i], embeddings[j])
        print(f"{i}-{j}: {sim}")